In [7]:
!pip install --upgrade "langchain>=0.2.0" "langchain-openai>=0.0.5" "langchain-community>=0.0.15"
!pip install --upgrade "singlestoredb>=1.0.0" "langchain-singlestoredb>=0.1.0"
!pip install --upgrade gradio requests python-dotenv
!pip install --upgrade sentence-transformers tiktoken

ERROR: Could not find a version that satisfies the requirement langchain-singlestoredb>=0.1.0 (from versions: none)
ERROR: No matching distribution found for langchain-singlestoredb>=0.1.0


In [9]:
import os
import gradio as gr
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.tools import Tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.vectorstores import SingleStoreDB
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from datetime import datetime
import requests
import json
import singlestoredb as s2

In [11]:
SINGLESTORE_HOST = "add your singlestore host url"  # e.g., "svc-12345-dml.aws-virginia-6.svc.singlestore.com"
SINGLESTORE_PORT = 3306
SINGLESTORE_USER = "admin"
SINGLESTORE_PASSWORD = "add your singlestore password"
SINGLESTORE_DATABASE = "AgentLang"
SINGLESTORE_TABLE = "ai_agent_knowledge"
OPENAI_API_KEY = "add your open ai api key"
NEWSAPI_API_KEY = "add your newsapi key"

In [12]:
class SingleStoreManager:
    def __init__(self):
        self.connection_string = f"mysql://{SINGLESTORE_USER}:{SINGLESTORE_PASSWORD}@{SINGLESTORE_HOST}:{SINGLESTORE_PORT}/{SINGLESTORE_DATABASE}"
        self.embeddings = OpenAIEmbeddings()
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        self.vectorstore = None
        self.setup_database()
    
    def setup_database(self):
        """Initialize the SingleStore database and create necessary tables"""
        try:
            # Create connection
            conn = s2.connect(
                host=SINGLESTORE_HOST,
                port=SINGLESTORE_PORT,
                user=SINGLESTORE_USER,
                password=SINGLESTORE_PASSWORD,
                database=SINGLESTORE_DATABASE
            )
            
            # Create table if it doesn't exist
            create_table_sql = f"""
            CREATE TABLE IF NOT EXISTS {SINGLESTORE_TABLE} (
                id VARCHAR(255) PRIMARY KEY,
                content TEXT,
                metadata JSON,
                embedding VECTOR(1536) NOT NULL,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                VECTOR INDEX (embedding)
            )
            """
            
            with conn.cursor() as cursor:
                cursor.execute(create_table_sql)
                conn.commit()
            
            conn.close()
            
            # Initialize vector store
            self.vectorstore = SingleStoreDB(
                embedding=self.embeddings,
                table_name=SINGLESTORE_TABLE,
                host=SINGLESTORE_HOST,
                port=SINGLESTORE_PORT,
                user=SINGLESTORE_USER,
                password=SINGLESTORE_PASSWORD,
                database=SINGLESTORE_DATABASE
            )
            
            print("✅ SingleStore database initialized successfully!")
            
        except Exception as e:
            print(f"❌ Error setting up SingleStore database: {str(e)}")
            self.vectorstore = None
    
    def add_knowledge(self, text: str, metadata: dict = None):
        """Add knowledge to the vector database"""
        try:
            if not self.vectorstore:
                return "Database not available"
            
            # Split text into chunks
            chunks = self.text_splitter.split_text(text)
            
            # Create documents
            documents = []
            for i, chunk in enumerate(chunks):
                doc_metadata = metadata or {}
                doc_metadata.update({
                    "chunk_index": i,
                    "total_chunks": len(chunks),
                    "added_at": datetime.now().isoformat()
                })
                documents.append(Document(page_content=chunk, metadata=doc_metadata))
            
            # Add to vector store
            self.vectorstore.add_documents(documents)
            
            return f"Successfully added {len(documents)} chunks to knowledge base"
            
        except Exception as e:
            return f"Error adding knowledge: {str(e)}"
    
    def search_knowledge(self, query: str, k: int = 3):
        """Search for relevant knowledge in the database"""
        try:
            if not self.vectorstore:
                return "Database not available"
            
            # Perform similarity search
            results = self.vectorstore.similarity_search(query, k=k)
            
            if not results:
                return "No relevant information found in knowledge base"
            
            # Format results
            formatted_results = "📚 Knowledge Base Results:\n\n"
            for i, doc in enumerate(results, 1):
                formatted_results += f"{i}. {doc.page_content}\n"
                if doc.metadata:
                    formatted_results += f"   Metadata: {doc.metadata}\n"
                formatted_results += "\n"
            
            return formatted_results
            
        except Exception as e:
            return f"Error searching knowledge: {str(e)}"
    
    def get_knowledge_stats(self):
        """Get statistics about the knowledge base"""
        try:
            conn = s2.connect(
                host=SINGLESTORE_HOST,
                port=SINGLESTORE_PORT,
                user=SINGLESTORE_USER,
                password=SINGLESTORE_PASSWORD,
                database=SINGLESTORE_DATABASE
            )
            
            with conn.cursor() as cursor:
                cursor.execute(f"SELECT COUNT(*) FROM {SINGLESTORE_TABLE}")
                count = cursor.fetchone()[0]
            
            conn.close()
            
            return f"Knowledge base contains {count} documents"
            
        except Exception as e:
            return f"Error getting stats: {str(e)}"

In [13]:
db_manager = SingleStoreManager()

/tmp/ipykernel_229/2252901541.py:44: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `SingleStoreVectorStore` implementation in `langchain_singlestore`. See <https://github.com/singlestore-labs/langchain-singlestore> for details about the new implementation.
  self.vectorstore = SingleStoreDB(


✅ SingleStore database initialized successfully!


In [16]:
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    api_key=OPENAI_API_KEY
)

def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculator(expression: str) -> str:
    """Evaluate a mathematical expression."""
    try:
        # Basic safety check
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Only basic mathematical operations are allowed"
        return str(eval(expression))
    except Exception as e:
        return f"Error calculating: {str(e)}"

def get_latest_news(query: str = "", category: str = "") -> str:
    """Get the latest news headlines."""
    if not NEWSAPI_API_KEY or NEWSAPI_API_KEY == "your-newsapi-key-here":
        return "⚠️ News API key not configured. Please set NEWSAPI_API_KEY."

    url = "https://newsapi.org/v2/top-headlines"
    params = {
        "apiKey": NEWSAPI_API_KEY,
        "language": "en",
        "pageSize": 5
    }

    if query:
        params["q"] = query
    if category and category.lower() in ["business", "entertainment", "general", "health", "science", "sports", "technology"]:
        params["category"] = category.lower()
    elif not query:
        params["category"] = "general"

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            news_data = response.json()
            if news_data["totalResults"] == 0:
                return get_location_news(query)

            result = f"📰 Latest News {f'on {query}' if query else ''} {f'in {category}' if category else ''}:\n\n"
            for i, article in enumerate(news_data["articles"], 1):
                result += f"{i}. {article['title']}\n"
                result += f"   📍 Source: {article['source']['name']}\n"
                result += f"   📅 Published: {article['publishedAt']}\n"
                result += f"   📝 Summary: {article['description'] if article['description'] else 'No description available'}\n"
                result += f"   🔗 URL: {article['url']}\n\n"

            return result
        else:
            return f"❌ Error fetching news: {response.status_code}"
    except Exception as e:
        return f"❌ Error processing news request: {str(e)}"

def get_location_news(location: str) -> str:
    """Get news for a specific location."""
    if not NEWSAPI_API_KEY or NEWSAPI_API_KEY == "your-newsapi-key-here":
        return "⚠️ News API key not configured. Please set NEWSAPI_API_KEY."

    url = "https://newsapi.org/v2/everything"
    params = {
        "apiKey": NEWSAPI_API_KEY,
        "q": location,
        "sortBy": "publishedAt",
        "language": "en",
        "pageSize": 5
    }

    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            news_data = response.json()

            if news_data["totalResults"] == 0:
                return f"ℹ️ No news found for location: {location}. Try a different search term."

            result = f"📰 Latest News related to {location}:\n\n"
            for i, article in enumerate(news_data["articles"], 1):
                result += f"{i}. {article['title']}\n"
                result += f"   📍 Source: {article['source']['name']}\n"
                result += f"   📅 Published: {article['publishedAt']}\n"
                result += f"   📝 Summary: {article['description'] if article['description'] else 'No description available'}\n"
                result += f"   🔗 URL: {article['url']}\n\n"

            return result
        else:
            return f"❌ Error fetching location news: {response.status_code}"
    except Exception as e:
        return f"❌ Error processing location news request: {str(e)}"

def add_to_knowledge_base(text: str, title: str = "", source: str = "") -> str:
    """Add information to the knowledge base."""
    metadata = {}
    if title:
        metadata["title"] = title
    if source:
        metadata["source"] = source
    metadata["type"] = "user_added"
    
    return db_manager.add_knowledge(text, metadata)

def search_knowledge_base(query: str) -> str:
    """Search the knowledge base for relevant information."""
    return db_manager.search_knowledge(query)

def get_knowledge_base_stats() -> str:
    """Get statistics about the knowledge base."""
    return db_manager.get_knowledge_stats()

print("✅ Agent tools defined successfully!")

✅ Agent tools defined successfully!


In [17]:
!pip install -U duckduckgo-search

In [18]:
duckduckgo_search = DuckDuckGoSearchRun()

# Define all tools
tools = [
    Tool(
        name="Search",
        func=duckduckgo_search.run,
        description="Useful for searching the web for current information."
    ),
    Tool(
        name="Calculator",
        func=calculator,
        description="Useful for performing mathematical calculations. Input should be a mathematical expression."
    ),
    Tool(
        name="CurrentTime",
        func=get_current_time,
        description="Get the current date and time. No input is needed."
    ),
    Tool(
        name="LatestNews",
        func=get_latest_news,
        description="Get the latest news headlines. You can specify a search query and/or category (business, entertainment, health, science, sports, technology)."
    ),
    Tool(
        name="LocationNews",
        func=get_location_news,
        description="Get news for a specific location or city. Input should be the name of the location (e.g., 'Mumbai', 'New York')."
    ),
    Tool(
        name="AddKnowledge",
        func=add_to_knowledge_base,
        description="Add information to the knowledge base. Input should be the text content you want to store."
    ),
    Tool(
        name="SearchKnowledge",
        func=search_knowledge_base,
        description="Search the knowledge base for relevant information. Input should be your search query."
    ),
    Tool(
        name="KnowledgeStats",
        func=get_knowledge_base_stats,
        description="Get statistics about the knowledge base. No input needed."
    )
]

In [20]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an intelligent assistant with access to a knowledge base and various tools.
    
    Your tools include:
    - Web search for current information
    - News retrieval (general and location-based)
    - Mathematical calculations
    - Current time
    - Knowledge base operations (add, search, stats)
    
    When users want to:
    - Store information: Use AddKnowledge tool
    - Find stored information: Use SearchKnowledge tool first, then web search if needed
    - Get knowledge base info: Use KnowledgeStats tool
    
    Always search your knowledge base first when users ask questions that might have been previously stored.
    If the knowledge base doesn't have relevant information, then use web search.
    
    Be helpful, accurate, and explain your reasoning clearly.
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create the agent
agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create the agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

# Initialize chat history
chat_history = []
print("✅ Agent created successfully!")

✅ Agent created successfully!


In [21]:
def process_input(message):
    """Process user input through the agent"""
    global chat_history
    try:
        # Run the agent
        response = agent_executor.invoke({
            "input": message,
            "chat_history": chat_history
        })
        
        # Update chat history
        chat_history.append(HumanMessage(content=message))
        chat_history.append(AIMessage(content=response["output"]))
        
        return response["output"]
    except Exception as e:
        error_msg = f"❌ Error processing request: {str(e)}"
        chat_history.append(HumanMessage(content=message))
        chat_history.append(AIMessage(content=error_msg))
        return error_msg

def add_knowledge_directly(text, title, source):
    """Direct function to add knowledge through the interface"""
    if not text.strip():
        return "⚠️ Please provide some text to add to the knowledge base."
    
    metadata = {}
    if title.strip():
        metadata["title"] = title.strip()
    if source.strip():
        metadata["source"] = source.strip()
    metadata["type"] = "direct_add"
    
    result = db_manager.add_knowledge(text, metadata)
    return result

print("✅ Helper functions ready!")

✅ Helper functions ready!


In [22]:
with gr.Blocks(title="AI Agent with SingleStore Database", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 AI Agent Dashboard with SingleStore Database")
    gr.Markdown("Ask me anything! I can search the web, get news, perform calculations, and manage a persistent knowledge base.")
    
    with gr.Tabs():
        # Chat Tab
        with gr.TabItem("💬 Chat"):
            chatbot = gr.Chatbot(height=500, placeholder="Start chatting with your AI agent...")
            msg = gr.Textbox(
                label="Your question", 
                placeholder="Ask me about stored knowledge, latest news, web search, or calculations...",
                lines=2
            )
            
            with gr.Row():
                submit_btn = gr.Button("Send", variant="primary")
                clear_btn = gr.Button("Clear Chat")
            
            # Example prompts
            gr.Markdown("### 💡 Example Commands:")
            with gr.Row():
                gr.Examples(
                    examples=[
                        "What's in my knowledge base?",
                        "Search knowledge base for AI trends",
                        "Add this to knowledge: Python is a programming language",
                        "What's the latest news in technology?",
                        "Calculate 15% of 250",
                        "What time is it?"
                    ],
                    inputs=msg
                )
        
        # Knowledge Management Tab
        with gr.TabItem("📚 Knowledge Base"):
            gr.Markdown("### Add Knowledge Directly")
            
            with gr.Row():
                with gr.Column():
                    knowledge_text = gr.Textbox(
                        label="Knowledge Content", 
                        placeholder="Enter the information you want to store...",
                        lines=5
                    )
                    knowledge_title = gr.Textbox(
                        label="Title (Optional)", 
                        placeholder="Give this knowledge a title..."
                    )
                    knowledge_source = gr.Textbox(
                        label="Source (Optional)", 
                        placeholder="Where did this information come from?"
                    )
                    add_knowledge_btn = gr.Button("Add to Knowledge Base", variant="primary")
                
                with gr.Column():
                    knowledge_result = gr.Textbox(
                        label="Result", 
                        lines=10,
                        interactive=False
                    )
            
            gr.Markdown("### Search Knowledge Base")
            with gr.Row():
                search_query = gr.Textbox(
                    label="Search Query", 
                    placeholder="What do you want to find in the knowledge base?"
                )
                search_btn = gr.Button("Search", variant="secondary")
            
            search_result = gr.Textbox(
                label="Search Results", 
                lines=10,
                interactive=False
            )
            
            # Stats section
            gr.Markdown("### Knowledge Base Statistics")
            stats_btn = gr.Button("Get Stats")
            stats_result = gr.Textbox(label="Statistics", interactive=False)

    # Event handlers
    def respond(message, chat_history):
        if not message.strip():
            return "", chat_history
        
        bot_message = process_input(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    def clear_chat():
        global chat_history
        chat_history = []
        return None

    def search_kb(query):
        if not query.strip():
            return "⚠️ Please enter a search query."
        return db_manager.search_knowledge(query)

    def get_stats():
        return db_manager.get_knowledge_stats()

    # Connect event handlers
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    submit_btn.click(respond, [msg, chatbot], [msg, chatbot])
    clear_btn.click(clear_chat, None, chatbot, queue=False)
    
    add_knowledge_btn.click(
        add_knowledge_directly, 
        [knowledge_text, knowledge_title, knowledge_source], 
        knowledge_result
    )
    search_btn.click(search_kb, search_query, search_result)
    stats_btn.click(get_stats, None, stats_result)

print("✅ Gradio interface created successfully!")

✅ Gradio interface created successfully!


/tmp/ipykernel_229/842703113.py:8: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=500, placeholder="Start chatting with your AI agent...")


In [23]:
if __name__ == "__main__":
    print("🚀 Starting AI Agent with SingleStore Database...")
    print("📊 Database Status:", "✅ Connected" if db_manager.vectorstore else "❌ Not Connected")
    print("🔑 API Status:")
    print(f"  - OpenAI: {'✅' if OPENAI_API_KEY != 'your-openai-api-key-here' else '❌'}")
    print(f"  - News API: {'✅' if NEWSAPI_API_KEY != 'your-newsapi-key-here' else '❌'}")
    print(f"  - SingleStore: {'✅' if SINGLESTORE_HOST != 'your-singlestore-host' else '❌'}")
    
    demo.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )

🚀 Starting AI Agent with SingleStore Database...
📊 Database Status: ✅ Connected
🔑 API Status:
  - OpenAI: ✅
  - News API: ✅
  - SingleStore: ✅
* Running on local URL:  http://0.0.0.0:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/05/26 06:09:17 [W] [service.go:132] login to server failed: dial tcp 44.237.78.176:7000: i/o timeout


In [24]:
def test_basic_functionality():
    """Test basic agent functionality without GUI"""
    print("🧪 Testing basic functionality...")
    
    # Test current time
    print("Time:", get_current_time())
    
    # Test calculator
    print("Calculator (2+2):", calculator("2+2"))
    
    # Test knowledge base
    if db_manager.vectorstore:
        print("Adding test knowledge...")
        result = db_manager.add_knowledge("Python is a programming language created by Guido van Rossum")
        print("Add result:", result)
        
        print("Searching knowledge...")
        search_result = db_manager.search_knowledge("Python programming")
        print("Search result:", search_result)
    else:
        print("⚠️ Database not available for testing")

def quick_chat_test():
    """Quick test of the agent without launching Gradio"""
    print("🧪 Testing agent chat...")
    test_message = "What time is it?"
    response = process_input(test_message)
    print(f"Q: {test_message}")
    print(f"A: {response}")

# Uncomment these lines to run tests:
# test_basic_functionality()
# quick_chat_test()